# Análisis Exploratorio de Datos (EDA) - Online Retail

En este cuaderno, realizaremos un **análisis paso a paso** de un dataset de facturación que contiene columnas como:
 - **InvoiceNo**: Número de factura
 - **StockCode**: Código del producto
 - **Description**: Descripción del producto
 - **Quantity**: Cantidad vendida (puede ser negativa en devoluciones)
 - **InvoiceDate**: Fecha de la factura
 - **UnitPrice**: Precio unitario de cada artículo
 - **CustomerID**: ID del cliente (puede haber nulos si no se registró)
 - **Country**: País donde se realizó la compra

El objetivo es **entender** el comportamiento de las ventas, los países más relevantes, los productos más comprados, y otras métricas de interés.

In [1]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
!unzip ecomerce_customers_segmentation.zip

--2025-09-12 17:32:11--  https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip [following]
--2025-09-12 17:32:12--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7548686 (7.2M) [application/zip]
Saving to: ‘ecomerce_customers_segmentation.zip’

ecomerce_customers_ 100%[===================>]

## 1. Importar librerías y cargar datos
Asegúrate de tener el archivo CSV en la misma carpeta, o ajusta la ruta de `pd.read_csv` según tu necesidad.

In [2]:
import pandas as pd
import numpy as np

# Carga del dataset (ajusta el nombre a tu archivo)
df = pd.read_csv('data.csv',encoding="latin-1") # Puedes necesitar encoding='latin1' si hay caracteres especiales
pd.set_option('display.max_columns', None)
# Mostramos las primeras filas para inspección
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Información general del dataset
Aquí veremos:
- Dimensiones del DataFrame
- Tipos de datos
- Presencia de valores nulos
- Estadísticos descriptivos de las columnas numéricas

In [3]:
df.shape

(541909, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [6]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


**Observación**: Algunas facturas pueden ser devoluciones si `Quantity` es negativa. Además, puede haber filas sin `CustomerID`.

## 3. Limpieza básica y transformación
### 3.1. Revisar y eliminar posibles filas con datos problemáticos
 - Filas donde `Description` esté vacía
 - Filas con `Quantity == 0` (puede haber poco sentido en un registro así)
 - Filas con `UnitPrice <= 0` (podría ser error o promoción anómala)
 - Convertir la columna `InvoiceDate` a formato fecha, para luego analizar tiempos

In [7]:
df.dropna(subset=['Description'])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [9]:
df = df[df['Quantity']!=0]

In [10]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [11]:
df = df[df['Quantity']>0]

In [12]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'],format='%m/%d/%Y %H:%M')

/tmp/ipython-input-2243013911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'],format='%m/%d/%Y %H:%M')


In [13]:
df['InvoiceDate'][0]

Timestamp('2010-12-01 08:26:00')

## 4. Creación de columna "TotalPrice"
Para conocer el valor total de cada línea de venta, calculamos `Quantity * UnitPrice`.

In [14]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

/tmp/ipython-input-523712759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


In [15]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


## 5. Análisis paso a paso
A continuación, veremos distintos análisis que **paso a paso** nos ayudarán a entender mejor el negocio.

### 5.1. ¿Cuántas transacciones totales hay y cuántos clientes diferentes?
Esto da un primer vistazo al tamaño del negocio en términos de facturas y base de clientes.

In [16]:
df.groupby('CustomerID').count()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalPrice
CustomerID,,,,,,,,
12346.0,1,1,1,1,1,1,1,1
12347.0,182,182,182,182,182,182,182,182
12348.0,31,31,31,31,31,31,31,31
12349.0,73,73,73,73,73,73,73,73
12350.0,17,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...
18280.0,10,10,10,10,10,10,10,10
18281.0,7,7,7,7,7,7,7,7
18282.0,12,12,12,12,12,12,12,12


### 5.2. ¿Cuáles son los países más frecuentes en las ventas?
Podemos contar cuántas líneas de factura vienen de cada país. Esto ayuda a entender nuestro alcance geográfico.

In [17]:
df.groupby('Country').count()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice
Country,,,,,,,,
Australia,1185,1185,1185,1185,1185,1185,1185,1185
Austria,398,398,398,398,398,398,398,398
Bahrain,18,18,18,18,18,18,17,18
Belgium,2031,2031,2031,2031,2031,2031,2031,2031
Brazil,32,32,32,32,32,32,32,32
Canada,151,151,151,151,151,151,151,151
Channel Islands,748,748,748,748,748,748,748,748
Cyprus,614,614,614,614,614,614,614,614
Czech Republic,25,25,25,25,25,25,25,25


> Tip: Si queremos ver el total en **monetario** por país, podemos agrupar por `Country` y sumar `TotalPrice`.

In [18]:
df.groupby('Country')['TotalPrice'].sum()

,TotalPrice
Country,
Australia,138521.310
Austria,10198.680
Bahrain,754.140
Belgium,41196.340
Brazil,1143.600
Canada,3666.380
Channel Islands,20450.440
Cyprus,13590.380
Czech Republic,826.740


### 5.3. ¿Cuáles son los productos más vendidos?
Aquí vamos a usar `Description` para ver cuántas líneas se han vendido (o la suma total de `Quantity`).

In [19]:
df.groupby('Description')['Quantity'].sum()

,Quantity
Description,
4 PURPLE FLOCK DINNER CANDLES,144
50'S CHRISTMAS GIFT BAG LARGE,1915
DOLLY GIRL BEAKER,2455
I LOVE LONDON MINI BACKPACK,389
I LOVE LONDON MINI RUCKSACK,1
...,...
wrongly coded 20713,800
wrongly coded 23343,1000
wrongly marked,48


### 5.4. ¿Cuál es el total de ingresos (revenue) y el ticket promedio?
 - El ingreso total es la suma de **TotalPrice**.
 - El ticket promedio podemos calcularlo como el promedio de **TotalPrice** dentro de cada factura (o línea).

In [20]:
df['TotalPrice'].sum()

np.float64(10644560.424)

In [21]:
df.groupby('InvoiceNo')['TotalPrice'].mean()

,TotalPrice
InvoiceNo,
536365,19.874286
536366,11.100000
536367,23.227500
536368,17.512500
536369,17.850000
...,...
581586,84.800000
581587,16.630000
A563185,11062.060000


> Para un ticket promedio a nivel de **factura**, debemos agrupar por `InvoiceNo`, sumar `TotalPrice` y luego promediar.

In [22]:
df.groupby('InvoiceNo')['TotalPrice'].sum().mean()

np.float64(513.5353350057893)

### 5.6. ¿Tenemos devoluciones?
Como `Quantity` puede ser negativa, **representa** devoluciones. Veamos cuántas hay y el impacto en valor.

In [23]:
df_devoluciones = df[df['Quantity']<0]

In [24]:
print(df_devoluciones)

Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, TotalPrice]
Index: []


### 5.7. ¿Cuántos clientes realizan compras repetidas?
Podemos ver cuántos clientes compran más de una vez.

In [25]:
df['CustomerID'].value_counts()

,count
CustomerID,
17841.0,7847
14911.0,5677
14096.0,5111
12748.0,4596
14606.0,2700
...,...
17986.0,1
16737.0,1
13017.0,1


## 6. Posibles siguientes pasos
- **Crear segmentaciones** (p. ej. RFM: Recency, Frequency, Monetary) para clasificar clientes.
- **Analizar descuentos/promos** si tuvieras columnas extra.
- **Fusionar** con datos de marketing para calcular CAC (Customer Acquisition Cost) y LTV (Lifetime Value), etc.
- **Explorar** variables estacionales o días de la semana con mayor venta.

Con este **análisis paso a paso**, se obtiene un primer panorama de:
 - Cantidad de facturas y clientes.
 - Productos más vendidos.
 - Ingresos por país.
 - Montos totales y tickets promedio.
 - Distribución mensual de ventas.
 - Devoluciones y recurrencia de clientes.

¡A medida que se adquieran más habilidades en Python y análisis de datos, se pueden añadir gráficos, segmentaciones avanzadas y modelos predictivos!